In [1]:
########################
#
# Excersises
#
########################


### E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?


In [2]:
#Why not both?
# possible trigrams =27*27*27 =19683 (too big to visuallize)


In [11]:
#By brain is missing some matmul insights so lets do counting model
# first

In [12]:
import requests 
url="https://raw.githubusercontent.com/karpathy/makemore/master/names.txt"
res=requests.get(url)
words=(res.text).splitlines()
print(len(words),max(len(w) for w in words),min(len(w) for w in words))

32033 15 2


In [13]:
import torch
import matplotlib.pyplot as plt 
import torch.nn.functional as F

In [73]:
chars =sorted(list(set((''.join(words)))))
# map char to int 
stoi={s:i+1 for i, s in enumerate(chars)}
stoi["."]=0
# map int to char 
itos={i:s for s,i in stoi.items()}

In [78]:
# calculate trigram frequencies in Matrix Form 
N=torch.zeros(27,27,27 ,dtype=int)
b={}

for w in words:
    chs=['.']+list(w)+['.']
    for ch1,ch2 ,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        ix3=stoi[ch3]
        N[ix1][ix2][ix3]+=1

In [190]:
# Convert Trigram frequency matrix to probability matrix
P=(N+1).float()# adding 1 to prevent probability from being 0 giving infinite loss
P=P/P.sum(2,keepdim=True)
print(P.shape)

torch.Size([27, 27, 27])


In [77]:
p.shape

torch.Size([28])

In [85]:
P[0][0]

tensor([3.7037e-02, 9.8990e-05, 3.8314e-03, 5.9172e-03, 1.9194e-03, 2.4295e-04,
        1.4085e-02, 1.5385e-02, 6.4350e-04, 3.5261e-04, 1.2987e-02, 5.3191e-03,
        6.2696e-04, 1.2255e-03, 1.1990e-03, 3.8081e-04, 1.3514e-02, 2.6316e-02,
        4.4723e-04, 1.9685e-03, 3.3784e-03, 7.3638e-04, 2.0040e-03, 8.4034e-03,
        1.5873e-02, 7.2833e-04, 2.9412e-03])

In [ ]:
p[0][0]

In [330]:
# sampling dumb lang  model basesd on probability matrix 
for i in range(10):

  ix=0
  ixx=0
  out=[]
  while True:
    p=P[ixx][ix]
    # print(itos[ixx],itos[ix])
    # to compare againet completely random 
    # p=N[ix].float()
    # p=p/p.sum()
    # p=torch.ones(27)/27
    ixx=ix
    ix= torch.multinomial(p,num_samples=1,replacement=True).item()
    out.append(itos[ix])
    if ix==0:
      break
  print(''.join(out))

xox.
ch.
.
don.
fer.
rah.
elloufkylah.
la.
di.
be.


In [199]:
# loss calculation for model based on probability materix 
logLikelihood=0.0
nll=0
n=0
for w in  words:
    chs=['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        ix3=stoi[ch3]
        prob= P[ix1][ix2][ix3]
        logprob=torch.log(prob)#multiplication of probabilites same as addition of negative log probabilites
        logLikelihood+=logprob
        n+=1  
    nll=-logLikelihood;
      
print(f"log {logLikelihood}")
print(f"nll {nll}")
print(f"LOSS nll/n {nll/n}")

log -410414.96875
nll 410414.96875
LOSS nll/n 2.092747449874878


In [ ]:
#NOW THE HARD PART TARGET:ACHIEVE LOWER LOSS THAN 2.5

In [15]:
# create training set 

xs,ys=[],[]
n=0
for w in words:
    chs=['.']+list(w)+['.']
    for ch1,ch2 ,ch2 in zip(chs,chs[1:],chs[2:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        ix3=stoi[ch2]
        # print(ch1,ch2) be carful with prints and for loops long for loops are very hard to stop ,kernel become un alive 
        xs.append([ix1,ix2])
        ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)
xs.shape,ys.shape

(torch.Size([196113, 2]), torch.Size([196113]))

In [412]:
# MAKING MODELLL
W=torch.ones(54,27,requires_grad=True).float()
xenc= F.one_hot(xs,num_classes=27).float().view(-1,54)

# forward pass


In [432]:

for k in range(1):
    # forward pass
    num=ys.nelement()
    logits= xenc@W
    
    counts=logits.exp()
    
    probs=counts/counts.sum(1,keepdims=True)
    # loss calculation
    
    loss=-probs[torch.arange(num),ys].log().mean() #+0.01*(W**2).mean()#regularization ie smoothing the weights
    
    # loss=F.cross_entropy(probs,ys)
    W.grad=None 
    loss.backward()
    print(k,loss)
    # a.append(loss.data)
    # update
    W.data+=-50*W.grad

0 tensor(0.0639, grad_fn=<NegBackward0>)


In [425]:
x=torch.tensor([[1,2,3],[1,2,3],[1,2,3]])

In [429]:
x.sum(1)

tensor([6, 6, 6])

In [336]:
ys[0]

tensor(13)